The skyline of a city is composed of several buildings of various widths and heights, possibly overlapping one another when viewed from a distance. We can represent the buildings using an array of (left, right, height) tuples, which tell us where on an imaginary x-axis a building begins and ends, and how tall it is. The skyline itself can be described by a list of (x, height) tuples, giving the locations at which the height visible to a distant observer changes, and each new height.

Given an array of buildings as described above, create a function that returns the skyline.

For example, suppose the input consists of the buildings [(0, 15, 3), (4, 11, 5), (19, 23, 4)]. In aggregate, these buildings would create a skyline that looks like the one below.

```
     ______  
    |      |        ___
 ___|      |___    |   | 
|   |   B  |   |   | C |
| A |      | A |   |   |
|   |      |   |   |   |
------------------------
```

As a result, your function should return [(0, 3), (4, 5), (11, 3), (15, 0), (19, 4), (23, 0)].

In [1]:
from typing import List
from heapq import heapify, heappop, heappush
from dataclasses import dataclass

In [2]:
@dataclass
class outline:
    loc: int
    height: int
    is_end_of_outline: bool

In [21]:
## This uses two dictionary, one to track begin and end of building out, the other 
## to track each location point to its height
## And uses a (min) heap that requires flipping number to make it a max heap to track
## active height of a building
## for time complexity, this should be O(n), where n is number of total location 
## points on the x-axis
## for space complexity, this should be O(n) too

def skyline(buildings: List[List[int]]) -> List[List[int]]:
    assert buildings

    hts = {} # pt: (height, if pt is end of a building outline)
    ends = {} # starting loc: ending loc
    
    # first, record heights and location
    for building in buildings:
        x, y, h = building
        hts[x] = (h, False)
        hts[y] = (h, True)
        ends[x] = y

    res = []
    active_height = [] # elements are tuples of (-height, height, ending pt); the first num of -height is to make min heap into a max heap and keep the sanity
    heapify(active_height)
    # second, scan from left x position and keep a heap of active heights
    for loc in sorted(hts.keys()):
        ht, is_end = hts[loc]
        if is_end:
            # pop from active heights if location point 
            while len(active_height) > 0 and active_height[0][2] <= loc:
                heappop(active_height)
            cur_max_ht = active_height[0][1] if len(active_height) > 0 else 0
            if ht > cur_max_ht:
                res.append((loc, cur_max_ht))
        else:
            cur_max_ht = active_height[0][1] if len(active_height) > 0 else 0
            if ht > cur_max_ht:
                res.append((loc, ht))
            heappush(active_height, (-ht, ht, ends[loc]))
    return res

In [24]:
'''tests'''

#      ______  
#     |      |        ___
#  ___|      |___    |   | 
# |   |   B  |   |   | C |
# | A |      | A |   |   |
# |   |      |   |   |   |
# ------------------------


test1 = [(0, 15, 3), (4, 11, 5), (19, 23, 4)]
exp1 = [(0, 3), (4, 5), (11, 3), (15, 0), (19, 4), (23, 0)]
out1 = skyline(test1)
assert out1 == exp1, f"expected: {exp1}, got: {out1}"


In [26]:
'''test2'''
#    _ _
#   |   |_
#   |     |__
#  _|        |   ____ 
#  |         |  |    | __
#  |         |  |        |
#  |         |  |        |
# # ------------------------------

test2 = [(2,9,10),(3,7,15),(5,12,12),(15,20,10),(19,24,8)]
exp2 = [(2,10),(3,15),(7,12),(12,0),(15,10),(20,8),(24,0)]
out2 = skyline(test2)
assert out2 == exp2, f"expected: {exp2}, got: {out2}"

In [ ]:
'''
psuedo:

Given any point to consider:
if it's starting of a building:
    if something else is taller:
        skip rendering
    if something is equal:
        skip
    if nothing is taller:
        render with pt and ht
if it's a ending pt:
    if something else is taller:
        skip
    if something else is equal:
        skip
    if something else is shorter:
        render with self and the next shorter ht




'''